# Countdown Numbers Game

![Countdown_titles_2012-removebg-preview](https://user-images.githubusercontent.com/55446533/167231820-53955fa6-a65c-4192-9462-39d728a2a819.png)

###  Overview and explanation of the Countdown Numbers Game


The Countdown Numbers Game is a popular arithmetic quiz which has been played as a two-player game on French and British television. [[6](#section6)] The contestants, who are competing against each other, select 6 numbers that are face down. There is an option of 24 numbers from which the 6 are picked from. The 24 tiles are split into 2 groups: *small numbers* and *large numbers*. The small numbers are a selection of numbers from **1** to **10**. There are 2 each of these. There are 4 large numbers, **25**, **50**, **75** and **100**. The contestants decide how many large and small numbers will be used in the 6. There is a maximum of 4 large numbers that can be used and a minimum of 0. For example, the contestant could pick 2 large numbers and 4 small numbers. The contestants then have to solve the equation.


<div><img src="https://user-images.githubusercontent.com/55446533/167232436-2d6416c0-570b-41ac-8bbd-d3a798809d5e.jpg" width=500> </div>

#### Rules [[5](#section5)]

* 6 numbers are chosen at random from a set of 24 which contains all numbers from *1* to *10* (***small numbers***) twice plus *25*, *50*, *75* and *100* (***large numbers***). 
* A 3-digit target number is chosen at random
* The contestants are given 30 seconds to get as close to the target number as possibile
* Only the four standard operations ($+$ $−$ $×$ $÷$) can be used. As soon as two numbershave been used to make a new one, they can’t be used again, but the new number found can be used. 
* At all times, the running total must be a non-negative integer


*Points are then awarded to each contestant based on how close they came to the target number.*

### Example
*Here's an example of the game in action. In this instance, one number was selected from the large set, and the rest from the small set.* [[1](#section1)]

$ \{ 50 , 8 , 3 , 7 , 2 , 10 \} $

*The randomly selected target was **556***.

*There are multiple ways to solve this. The smallest solution requires just four numbers:*

$ (50 × 10) + (8 × 7) = 556 $

*More complex solutuion:*

$ (((50 - 7) × 3) + 10) × 8) ÷ 2 = 556 $

### Discussion of the complexity of the Countdown Numbers Game

*During the research for this project, I was trying to find an answer for the following qustions:*

* *Is it possible*
* *What are possible ways of doing it if it can be done* 
* *If it's more than one way, can you find different ways*
* *What is the ‘simplest’ solution to every problem (If there are multiple ways to solve any particular problem, what is the solution that requires the smallest number of donor numbers)*


##### The essence of the problem



*Given a sequence of source numbers and a single target number, attempt to construct an arithmetic expression using each of the source numbers at most once, and such that the result of evaluating the expression is the target number. The given numbers are restricted to being non-zero naturals, as are the intermediate results during evaluation of the expression, which can otherwise be freely constructed using addition, subtraction, multiplication and division.* [[7](#section7)]

Due to many different scenarios and variables available, the Countdown number game is extremely complex. Another way explaining the complexity of numbers game is the fact that generated *target* number sometimes creates scenarios where there is no possible way to correctly calculate it. The amount of ways in which the target number can be calculated adds on to the complexity of the game. There may not be just one correct answer but multiple ways of calculating it are possible which means thats finding all possible ways can be complex and time consuming. The player has only *30 seconds* to solve the equation which increases the difficulty. 

# `Reverse Polish Notation`
**RPN** - is a mathematical notation in which operators follow their operands [[2](#section2)]
***

In [1]:
# Reverse Polish Notation
# 3 4 5 + *

In [2]:
def evaluate_rpn(rpn):
    # A stack
    stack = []
    # Loop through rpn one item at the time
    for i in rpn:
        # Check if i is an integer
        if isinstance(i,int):
            # Append to the stack
            stack = stack + [i]
        else:
            # Pop from stack twice
            right = stack[-1]
            stack = stack[:-1]
            left = stack[-1]
            stack = stack[:-1]
            # Push operator applied to stack element
            stack = stack +[i(left,right)]
        # Should only be 1 item on stack
    return stack[0]         

## Solving the Countdown Numbers Game

In [3]:
import random

def subtract(x, y):
    return x-y

def add(x, y):
    if x <= y:
        return x+y
    raise ValueError

def multiply(x, y):
    if x <= y or x == 1 or y == 1:
        return x*y
    raise ValueError

def divide(x, y):
    if not y or x % y or y == 1:
        raise ValueError
    return x/y

# Represent arithmetic functions as a symbols
add.str = '+'
multiply.str = '*'
subtract.str = '-'
divide.str = '/'

OPERATORS = [add, subtract, multiply, divide]

In [4]:
def small_random():
    # Generates 4 small numbers between 1-10 at random
    return random.randint(1,10)

In [5]:
def large_random():
    # Generates 2 large numbers from given list at random
    return [25, 50, 75, 100][random.randint(0,2)]

In [6]:
def generate_rpn(numbers, depth=0):
    """Generates all permuations of RPN expression for given numbers"""
    for i in range(len(numbers)):
        yield ([numbers[i]], numbers[:i]+numbers[i+1:], numbers[i])
    if len(numbers) >= 2+depth:
        for rhs, rrs, rv in generate_rpn(numbers, depth+1):
            for lhs, lrs, lv in generate_rpn(rrs, depth):
                # Loop over the operators
                for op in OPERATORS:
                    try:
                        yield ([lhs, rhs, op], lrs, op(lv, rv))
                    except ValueError:
                        pass

In [7]:
def find_first_or_closest(target, numbers):
    """Find the first matching expression or the closest"""
    generate = generate_rpn(numbers)
    expression1, s, value1 = generate.__next__()
    if value1 == target:
        return expression1, target
    # Closest expression & value
    closest_v = value1
    closest_e = expression1

    for expression, s, value in generate_rpn(numbers):
        # Target we've been looking for
        if value == target:
            return expression, target
        # If no target value found, return closest to the target
        elif abs(target - value) < abs(target - closest_v):
            closest_v = value
            closest_e = expression
    return closest_e, closest_v

In [8]:
def flatten_expression(expression):
    """The recursive approach of flattening a raw expression to a single array
    https://miguendes.me/python-flatten-list#the-recursive-approach"""
    if len(expression) == 3:
        for y in flatten_expression(expression[0]):
            yield y
        for x in flatten_expression(expression[1]):
            yield x
        yield expression[2].str
    elif len(expression) == 1:
        yield expression[0]

In [9]:
# Idea of how to convert postifx to infix implemented from here https://codereview.stackexchange.com/questions/190533/countdown-numbers-game-solution-generator
def postfix_to_infix(iterable):
    """Converts RPN to human readable infix"""
    class Intermediate:
        def __init__(self, expr, priority=False):
            self.expr = expr
            self.priority = priority
    # Create a stack
    stack = []
    # Loop over the iterable
    for token in iterable:
        if token == "+" or token == "-":
            right = stack.pop()
            left = stack.pop()
            temp = Intermediate(left.expr + token + right.expr, True)
            stack.append(temp)
        elif token == "*" or token == "/":
            right = stack.pop()
            if right.priority:
                right = Intermediate("(" + right.expr + ")")
            left = stack.pop()
            if left.priority:
                left = Intermediate("(" + left.expr + ")")
            temp = Intermediate(left.expr + token + right.expr)
            stack.append(temp)
        else:
            stack.append(Intermediate(str(token)))
    return stack.pop().expr

In [10]:
def solve(target, numbers):
    expression, value = find_first_or_closest(target, numbers)
    solution = postfix_to_infix(flatten_expression(expression))
    return solution, value

In [11]:
""" Solves the countdown game """
def countdown_game(target, numbers):
        print("Target:", target)
        print("Numbers:", numbers)
        
        solution, value = solve(target, numbers)     
        # Find the first correct solution
        if value == target:
            print("Found solution:", solution, "=", value)
        else:
            print("Found closest:", solution, "=", value)

In [12]:
target = random.randint(100,1000)

In [13]:
numbers = [ large_random() ] + [ small_random() for i in range(5) ]

In [14]:
countdown_game(target,numbers)

Target: 421
Numbers: [50, 5, 5, 7, 1, 4]
Found solution: (5*(5*7+50)-4)*1 = 421
Took 0.8348808288574219 seconds.



## Explanation of the functional aspects of the code

*I've broke down the countdown game problem into smaller pieces. Each function is specified to perform one thing which increases *modularity*. This type of design is easier to test and debug.*

#### Iterators

An iterator is an object representing a stream of data; this object returns the data one element at a time. A Python iterator supports method called `__next__()`

In [3]:
def find_first_or_closest(target, numbers):
    generate = generate_rpn(numbers)
    expression1, s, value1 = generate.__next__()

*Iterator has been used to iterate over the generated permuations of RPN expression.*

#### Use of built- in functions

Python has a set of built-in functions. During the development of this project I took advantage of few functions, *len()*, *abs()* and *range()*.

* *len()* - Returns a length of an object
* *abs()* - Makes any negative number to positive
* *range()* - Returns a sequence of numbers, starting from 0 by default, and increments by 1 (by default)

*Example of len() function in the code, which is used to loop over all given numbers*

`for i in range(len(numbers)):`

Another example is a use of *range()* function which was helpful with generating the 6 numbers required for the game

In [8]:
numbers = [ large_random() ] + [ small_random() for i in range(5) ]

NameError: name 'large_random' is not defined

#### Generators

*Generators are a special class of functions that simplify the task of writing iterators. Regular functions compute a value and return it, but generators return an iterator that returns a stream of values.*

*Example in code*

In [9]:
def generate_rpn(numbers, depth=0):
    """Generates all permuations of RPN expression for given numbers"""
    for i in range(len(numbers)):
        yield ([numbers[i]], numbers[:i]+numbers[i+1:], numbers[i])

*Note the `yield` keyword, any function containing this keyword is a generator function.*

## References
***

<a id='section1'></a>[[1] Countdown Game Example](https://datagenetics.com/blog/august32014/index.html)

<a id='section2'></a>[[2] Reverse Polish Notation ](https://en.wikipedia.org/wiki/Reverse_Polish_notation)

<a id='section3'></a>[[3] Functional Programming in Python ](https://realpython.com/python-functional-programming/)

<a id='section4'></a>[[4] Stackoverflow: Countdown Binary Trees](https://stackoverflow.com/a/54496061)

<a id='section5'></a>[[5] (The Final) Countdown](https://easychair.org/publications/open/2L76)

<a id='section6'></a>[[6] Countdown Numbers Game: Solved, Analysed, Extended](http://doc.gold.ac.uk/aisb50/AISB50-S02/AISB50-S2-Colton-paper.pdf)

<a id='section7'></a>[[7] Countdown Problem](http://www.cs.nott.ac.uk/~pszgmh/countdown.pdf)